# Power Calculations

In [1]:
import cPickle
import os

import gdpy
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects import r
import scipy.stats as stats
import seaborn as sns
plt.rcParams['font.sans-serif'] = ['Helvetica'] + plt.rcParams['font.sans-serif']

from IPython.display import display, HTML

%matplotlib inline
%load_ext rpy2.ipython

outdir = '../output/power_calculations'
if not os.path.exists(outdir):
    os.makedirs(outdir)
private_outdir = '../private_output/power_calculations'
if not os.path.exists(private_outdir):
    os.makedirs(private_outdir)

In [2]:
# Read Gencode gene annotation table.
fn = '/srv/gsfs0/projects/rivas/data/gencode.v19_20170302/parsing/gencode.v19.annotation.table.tsv'
if not os.path.exists(fn):
    fn = '/oak/stanford/groups/mrivas/public_data/gencode.v19_20170526/parsing/gencode.v19.annotation.table.tsv'
gtable = pd.read_table(fn, index_col=0)
gtable['gencode_id'] = gtable.index
gtable.index = [x.split('.')[0] for x in gtable.index]

# Read LoF gene table.
lof_gene = pd.read_table('../output/lof_enrichment_analysis/lof_gene.tsv',
                         index_col=0)

## Power Calculations

I'm going to calculate the power to detect an LoF association for each gene in 
each population under a particular experimental design. Then I will
add annotation for the genes. I'm going to drop the `OTH` population because
it's not represenative of any real population.

In [3]:
fn = os.path.join(outdir, 'lof_gene_power.tsv')
if not os.path.exists(fn):
    pops = [x[3:] for x in lof_gene.columns if x[0:3] == 'AF_']
    for pop in pops:
        for rr in [2, 4]:
            power = []
            for gene in lof_gene.index:
                af = lof_gene.ix[gene, 'AF_{}'.format(pop)]
                if af == 0:
                    power.append(0)
                else:
                    power.append(gdpy.gpc_default(af, rr, 0.1, 25000, 0.5, 
                                                  alpha=2e-6, unselected=True))
            lof_gene['power_rr{}_{}'.format(rr, pop)] = power
    lof_gene.to_csv(fn, sep='\t')